In [0]:
catalog_name = "dmc_tallerfinaldatabricks_mariovento"

schema_silver = "silver"
schema_gold = "gold"

In [0]:
from pyspark.sql.functions import col, to_date, current_timestamp
from pyspark.sql.types import DecimalType, LongType

In [0]:
silver_ventas = spark.table(f"{catalog_name}.{schema_silver}.ventas")

# Carga las tablas de dimensiones desde el esquema gold.
dim_cliente = spark.table(f"{catalog_name}.{schema_gold}.dim_cliente")
dim_tienda = spark.table(f"{catalog_name}.{schema_gold}.dim_tienda")
dim_producto = spark.table(f"{catalog_name}.{schema_gold}.dim_producto")
dim_tiempo = spark.table(f"{catalog_name}.{schema_gold}.dim_tiempo")

# Realiza joins entre la tabla de ventas y las dimensiones para obtener las claves sustitutas (sk) y enriquecer los datos.
fact_ventas = (silver_ventas.alias("v")
    .join(dim_cliente.alias("c"), col("v.id_cliente") == col("c.id_cliente"), "left")
    .join(dim_tienda.alias("t"), col("v.id_tienda") == col("t.id_tienda"), "left")
    .join(dim_producto.alias("p"), col("v.id_producto") == col("p.id_producto"), "left")
    .join(dim_tiempo.alias("d"), to_date(col("v.fecha_venta")) == col("d.fecha"), "left")
    # Selecciona y transforma las columnas necesarias para la tabla de hechos.
    .select(
        col("c.sk_cliente"),
        col("t.sk_tienda"),
        col("p.sk_producto"),
        col("d.sk_tiempo"),
        col("v.id_venta").cast(LongType()),
        col("v.cantidad"),
        col("v.monto_corregido").cast(DecimalType(18,2)).alias("monto"),
        col("v.estado_monto").alias("estado"),
        col("v.fecha_venta"),
        current_timestamp().alias("creation_date")
    )
)

In [0]:
fact_ventas.write.format("delta").mode("overwrite").saveAsTable(f"{catalog_name}.{schema_gold}.fact_ventas")

In [0]:
display(fact_ventas)

In [0]:
%sql
-- OPTIMIZE reorganiza los archivos de la tabla para mejorar el rendimiento de las consultas.
-- ZORDER BY (sk_cliente, sk_producto) organiza los datos físicamente para acelerar búsquedas por estas columnas.
OPTIMIZE dmc_tallerfinaldatabricks_mariovento.gold.fact_ventas
ZORDER BY (sk_cliente, sk_producto);

-- VACUUM elimina archivos antiguos no referenciados, liberando espacio en disco.
-- RETAIN 168 HOURS indica que se retendrán archivos de los últimos 7 días antes de ser eliminados.
VACUUM dmc_tallerfinaldatabricks_mariovento.gold.fact_ventas RETAIN 168 HOURS; -- 7 días

In [0]:
%sql
describe history dmc_tallerfinaldatabricks_mariovento.gold.fact_ventas